In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Q1

Write a function that generates a random vector  V of length one million and multiplies it by a constant number (for example 5). Implement it with a for‑loop and also a vectorized implementation, and compare the execution time of both.

In [ ]:
# part a: for-loop implementation
def rand_vecf():
    V = torch.rand(1000000)
    for i in range(len(V)):
        V[i] *= 5
    return V

In [4]:
%%timeit
rand_vecf()

11.1 s ± 904 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# part b: vectorized implementation
def rand_vecV():
    V = torch.rand(1000000)
    return V*5

In [7]:
%%timeit
rand_vecV()

9.87 ms ± 733 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
